In [92]:
# Not subclassing prevents using different behaviour in training and in generation. Means dropout is not an option.

In [93]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
import scipy
from sklearn import preprocessing

%matplotlib inline

In [94]:
# Get in dataset
attacks = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/wustl_attacks.csv", sep=",")

In [95]:
attacks = preprocessing.normalize(attacks.to_numpy(), norm="max", axis=0)

In [96]:
seq_length = 10
batch_size = 64
num_features = attacks.shape[1]
codings_size = 100

In [97]:
# train_dataset = (tf.data.Dataset.from_tensor_slices(attacks[:9984]).shuffle(10000).batch(batch_size))
# test_dataset = attacks[attacks.shape[0]-5000:]

In [98]:
# should probably add batch and shuffle

train_dataset = attacks[:int(np.floor(attacks.shape[0]*4/5))]
test_dataset = attacks[int(np.floor(attacks.shape[0]*4/5)):]

# buffer_size = 10000
# batch_size = 50

# train_dataset = (tf.data.Dataset.from_tensor_slices(attacks[:int(np.floor(attacks.shape[0]*3/4))]).shuffle(buffer_size).batch(batch_size))
# test_dataset = (tf.data.Dataset.from_tensor_slices(attacks[int(np.floor(attacks.shape[0]*3/4)):]).shuffle(buffer_size).batch(batch_size))

In [99]:
train_dataset.shape, test_dataset.shape

((69612, 28), (17404, 28))

In [100]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return tf.keras.backend.random_normal(tf.shape(log_var)) * tf.keras.backend.exp(log_var / 2) + mean

In [101]:
# encoder

inputs = tf.keras.layers.Input(shape=[num_features,1])
z = tf.keras.layers.Flatten()(inputs)
z = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(z)
z = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(z)
codings_mean = tf.keras.layers.Dense(codings_size)(z) # mu
codings_log_var = tf.keras.layers.Dense(codings_size)(z) # gamma
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = tf.keras.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

In [102]:
# decoder

decoder_inputs = tf.keras.layers.Input(shape=[codings_size])
x = tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal")(decoder_inputs)
x = tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal")(x)
x = tf.keras.layers.Dense(num_features*1, activation="sigmoid")(x)
outputs = tf.keras.layers.Reshape([num_features, 1])(x)
variational_decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[outputs])

In [103]:
# VAE model

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = tf.keras.Model(inputs=[inputs], outputs=[reconstructions])

In [104]:
# losses

latent_loss = -0.5 * tf.keras.backend.sum(1 + codings_log_var - tf.keras.backend.exp(codings_log_var) - tf.keras.backend.square(codings_mean), axis=-1)
variational_ae.add_loss(tf.keras.backend.mean(latent_loss / float(num_features)))
# variational_ae.add_loss(tf.keras.backend.mean(latent_loss))
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-5)
variational_ae.compile(loss="binary_crossentropy", optimizer=optimizer)

In [105]:
history = variational_ae.fit(train_dataset, train_dataset, epochs=50, batch_size=64)

Epoch 1/50


1088/1088 [==============================] - 2s 963us/step - loss: 0.7922
Epoch 2/50
1088/1088 [==============================] - 1s 956us/step - loss: 0.4520
Epoch 3/50
1088/1088 [==============================] - 1s 951us/step - loss: 0.4034
Epoch 4/50
1088/1088 [==============================] - 1s 962us/step - loss: 0.3807
Epoch 5/50
1088/1088 [==============================] - 1s 958us/step - loss: 0.3579
Epoch 6/50
1088/1088 [==============================] - 1s 961us/step - loss: 0.3397
Epoch 7/50
1088/1088 [==============================] - 1s 949us/step - loss: 0.3295
Epoch 8/50
1088/1088 [==============================] - 1s 960us/step - loss: 0.3248
Epoch 9/50
1088/1088 [==============================] - 1s 989us/step - loss: 0.3208
Epoch 10/50
1088/1088 [==============================] - 1s 973us/step - loss: 0.3191
Epoch 11/50
1088/1088 [==============================] - 1s 961us/step - loss: 0.3180
Epoch 12/50
1088/1088 [==============================] - 1s 965us/step - l

In [106]:
predictions = variational_ae.predict(test_dataset)

num_values = test_dataset.shape[0] * test_dataset.shape[1]

scipy.stats.ks_2samp(test_dataset.reshape(num_values,), predictions.reshape(num_values,))

544/544 [==============================] - 0s 602us/step


KstestResult(statistic=0.33989928423679283, pvalue=0.0, statistic_location=0.00020024028834601522, statistic_sign=1)

In [107]:
scipy.stats.ks_2samp(test_dataset.reshape(num_values,), train_dataset.reshape(train_dataset.shape[0] * train_dataset.shape[1]))

KstestResult(statistic=0.03628791481496141, pvalue=0.0, statistic_location=0.8, statistic_sign=-1)

In [108]:
# df = pd.DataFrame(predictions.reshape(predictions.shape[0], predictions.shape[1]))
# df.to_csv("vae_swat_network.csv", index=False)